In [1]:
import gurobipy as gp

In [2]:
def le_dados(nome_arq):
    with open(nome_arq, 'r') as f:
        linhas = f.readlines()#ler linhas
        valores = linhas[0].strip().split(' ')
        qtd_cidades = int(valores[0])
        #Vetores de x e y
        xi = list()
        yi = list()
        del(linhas[0])
        for linha in linhas:
            valores = linha.strip().split(' ')
            xi.append(int(valores[0]))
            yi.append(int(valores[1]))

    #retornar dados
    return qtd_cidades, xi,  yi

In [3]:
def dist(x, y, cidades):
    from math import sqrt
    custo = list()
    b = list()
    for i, xi in enumerate(x):
        for j, yi in enumerate(y):
            if j == 0:
                a = (x[j] - x[i]) ** 2 + (y[j] - y[i]) ** 2
                b.append(sqrt(a))
            if j != cidades - 1:
                a = (x[j + 1] - x[i]) ** 2 + (y[j + 1] - y[i]) ** 2
                b.append(sqrt(a))
        custo.append(b[:])
        b.clear()
    print()
    return custo

In [4]:
def solve_CaixeiroViajante(nome_arq):
    #ler dados
    cidades, x, y = le_dados(nome_arq)

    mat_custos = dist(x, y, cidades)

    # Indices dos pontos de origem e destino
    origens = [i + 1 for i in range(cidades)]
    destinos = [i + 1 for i in range(cidades)]

    # Dicionário dos custos
    custos = dict()
    for i, origem in enumerate(origens):
        for j, destino in enumerate(destinos):
            custos[origem, destino] = mat_custos[i][j]

    # iniciar o modelo

    m = gp.Model()

    # variaveis de decisão
    x = m.addVars(origens, destinos, vtype=gp.GRB.BINARY)
    u = m.addVars(origens[1:], vtype=gp.GRB.INTEGER, ub = cidades - 1)

    # Função Obj

    m.setObjective(x.prod(custos), sense=gp.GRB.MINIMIZE)

    # restrições de origem
    c1 = m.addConstrs(
        gp.quicksum(x[i, j] for j in destinos if i != j) == 1
        for i in origens)

    # restrições de destino
    c2 = m.addConstrs(
        gp.quicksum(x[i, j] for i in origens if i != j) == 1
        for j in destinos)

    # restrições de eliminação de subrotas(MTZ)

    c3 = m.addConstrs(
        u[i] - u[j] + cidades * x[i, j] <= cidades - 1
        for i in origens[1:] for j in destinos[1:] if i != j
    )
    

    m.optimize()
    # Constrói o vetor com o circuito,
    circuito = [1]
    anterior = 1
    for ponto in range(cidades):
        for j in destinos:
            if round(x[anterior, j].X) == 1:
                circuito.append(j)
                anterior = j
                break
    return m.objVal,circuito, 

In [5]:
arq = 'TSP.txt'
resultado, circuito = solve_CaixeiroViajante(arq)
print(resultado)
print(circuito)


Using license file C:\Users\marya\gurobi.lic


GurobiError: License expired 2021-12-07